### Quickstart - Dicas básicas para entendimento e início de desenvolvimento

**Conectando ao EPM Webserver**

In [13]:
import epmwebapi as epm
import numpy as np
import datetime

#crie o objeto de conexão informando os endereços do EPM Webserver(Authentication Port e WEB API Port), usuário e senha.
connection = epm.EpmConnection('http://localhost:44333', 'http://localhost:44332', 'sa', 'psw')


**Leitura de dados brutos(Raw)**



In [14]:
bvname = 'ADM_Temperature'

bv = connection.getDataObjects([bvname])

end_date = datetime.datetime.now()
ini_date= end_date - datetime.timedelta(hours=1)
queryPeriod = epm.QueryPeriod(ini_date,end_date)

data = bv[bvname].historyReadRaw(queryPeriod)

print(data)


[(24.4, datetime.datetime(2018, 10, 11, 9, 53, 38, tzinfo=datetime.timezone.utc), 0)
 (24.3, datetime.datetime(2018, 10, 11, 9, 53, 58, tzinfo=datetime.timezone.utc), 0)
 (24.4, datetime.datetime(2018, 10, 11, 9, 54, 18, tzinfo=datetime.timezone.utc), 0)
 (24.5, datetime.datetime(2018, 10, 11, 10, 0, 38, tzinfo=datetime.timezone.utc), 0)
 (24.6, datetime.datetime(2018, 10, 11, 10, 8, 38, tzinfo=datetime.timezone.utc), 0)
 (24.7, datetime.datetime(2018, 10, 11, 10, 15, 18, tzinfo=datetime.timezone.utc), 0)
 (24.8, datetime.datetime(2018, 10, 11, 10, 22, 59, tzinfo=datetime.timezone.utc), 0)
 (24.9, datetime.datetime(2018, 10, 11, 10, 28, 59, tzinfo=datetime.timezone.utc), 0)
 (25. , datetime.datetime(2018, 10, 11, 10, 36, 19, tzinfo=datetime.timezone.utc), 0)
 (25.1, datetime.datetime(2018, 10, 11, 10, 43, 39, tzinfo=datetime.timezone.utc), 0)
 (25.2, datetime.datetime(2018, 10, 11, 10, 49, 19, tzinfo=datetime.timezone.utc), 0)]


**Leitura usando agregação de dados (Baseados na norma OPC UA)**
Para informações sobre os tipos de agregações, consulte o Apêndice A do Guia do Usuário: https://github.com/elipsesoftware/epmprocessor/blob/master/guiadousuario/Agregacoes.md

In [15]:
bvname = 'ADM_Temperature'

bv = connection.getDataObjects([bvname])

end_date = datetime.datetime.now()
ini_date= end_date - datetime.timedelta(hours=1)

queryPeriod = epm.QueryPeriod(ini_date,end_date)

processInterval = datetime.timedelta(seconds=300)
aggDetails = epm.AggregateDetails(processInterval, epm.AggregateType.Interpolative)


data = bv[bvname].historyReadAggregate(aggDetails, queryPeriod)

print(data)

[(24.396067, datetime.datetime(2018, 10, 11, 9, 53, 23, 841961, tzinfo=tzutc()), 0)
 (24.464695, datetime.datetime(2018, 10, 11, 9, 58, 23, 841961, tzinfo=tzutc()), 0)
 (24.53455 , datetime.datetime(2018, 10, 11, 10, 3, 23, 841961, tzinfo=tzutc()), 0)
 (24.597052, datetime.datetime(2018, 10, 11, 10, 8, 23, 841961, tzinfo=tzutc()), 0)
 (24.671461, datetime.datetime(2018, 10, 11, 10, 13, 23, 841961, tzinfo=tzutc()), 0)
 (24.740313, datetime.datetime(2018, 10, 11, 10, 18, 23, 841961, tzinfo=tzutc()), 0)
 (24.8069  , datetime.datetime(2018, 10, 11, 10, 23, 23, 841961, tzinfo=tzutc()), 0)
 (24.890234, datetime.datetime(2018, 10, 11, 10, 28, 23, 841961, tzinfo=tzutc()), 0)
 (24.960192, datetime.datetime(2018, 10, 11, 10, 33, 23, 841961, tzinfo=tzutc()), 0)
 (25.028374, datetime.datetime(2018, 10, 11, 10, 38, 23, 841961, tzinfo=tzutc()), 0)
 (25.096556, datetime.datetime(2018, 10, 11, 10, 43, 23, 841961, tzinfo=tzutc()), 0)
 (25.183777, datetime.datetime(2018, 10, 11, 10, 48, 23, 841961, tzin

**Escrita de conjuntos de dados históricos no EPM Server**


In [ ]:
import epmwebapi as epm
import numpy as np

bvname = 'bvname'

connection = epm.EpmConnection('http://localhost:44333', 'http://localhost:44332', 'sa', 'Epm2017')

bv = connection.getDataObjects([bvname])

#some values for write tests
newvalues = [50,60,30,40,10]
base = datetime.datetime(2018, 1, 1)
newdates = np.array([base + datetime.timedelta(hours=i) for i in range(5)])

# epm ndarray data format.
desc = np.dtype([('Value', '>f8'), ('Timestamp', 'object'), ('Quality', '>i4')])
datatemp = np.empty(len(newvalues), dtype=desc)

#loop to populate the object before send to EPM
i=0
while i < len(newvalues):
    datatemp['Value'][i] = newvalues[i]
    datatemp['Timestamp'][i] = newdates[i]
    datatemp['Quality'][i] = 0
    i = i+1
    
bv[bvname].historyUpdate(datatemp)

**Escrita de valores no EPM Server**

In [ ]:
import epmwebapi as epm

bvname = 'bvname'

connection = epm.EpmConnection('http://localhost:44333', 'http://localhost:44332', 'sa', 'Epm2017')

bv = connection.getDataObjects([bvname])

date = datetime.datetime.now()
value = 100
quality = 0 #zero is Good in OPC UA


bv[bvname].write(value, date, quality)


**Filtro de variáveis**

In [19]:
from epmwebapi.dataobjectattributes import DataObjectAttributes
from epmwebapi.dataobjectsfilter import DataObjectsFilter
from epmwebapi.dataobjectsfilter import DataObjectsFilterType
from epmwebapi.domainfilter import DomainFilter

Filter = DataObjectsFilter(DataObjectsFilterType.BasicVariable, '*Temperature*', None, None, DomainFilter.Continuous)

properties = DataObjectAttributes.Description | DataObjectAttributes.LowLimit | DataObjectAttributes.HighLimit | DataObjectAttributes.EU

query = connection.filterDataObjects(doFilter, properties)

#this query returns an OrderedDict
print(query)


OrderedDict([('ADM_Temperature', <epmwebapi.epmdataobject.EpmDataObject object at 0x0000018D459444A8>), ('TI_Temperature', <epmwebapi.epmdataobject.EpmDataObject object at 0x0000018D45944AC8>), ('TechnicalSupportTemperature', <epmwebapi.epmdataobject.EpmDataObject object at 0x0000018D459442E8>), ('E3Dev_Temperature', <epmwebapi.epmdataobject.EpmDataObject object at 0x0000018D459440F0>), ('EPMDev_Temperature', <epmwebapi.epmdataobject.EpmDataObject object at 0x0000018D45944358>), ('MeetingRoom11th_Temperature', <epmwebapi.epmdataobject.EpmDataObject object at 0x0000018D459442B0>), ('MeetingRoom10th_Temperature', <epmwebapi.epmdataobject.EpmDataObject object at 0x0000018D459441D0>), ('PowerDev_Temperature', <epmwebapi.epmdataobject.EpmDataObject object at 0x0000018D459443C8>), ('TIServidores_Temperature', <epmwebapi.epmdataobject.EpmDataObject object at 0x0000018D3DFB07B8>), ('Auditorio_Temperature', <epmwebapi.epmdataobject.EpmDataObject object at 0x0000018D4595DDA0>), ('Treinamento_Tem